In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
data_dir = '/home/aman/workspace/pandemic_data/datasets1/'

In [3]:
!ls -1 $data_dir/

citation_data.rtf
data_dictionary_linelist.csv
gh_mexico_linelist.csv
gisaid_metadata.tsv


In [4]:
# The files we will be importing


genomic_file  = os.path.join(data_dir, 'gisaid_metadata.tsv')

## Data Dictionary, prep

In [5]:
"""
Sample the TSV file!

We don't have a data dictionary, so we'll first import a small number of rows, 
figure out what we want, and then do a bigger import.

GISAID provides TSV tab delimited format. 
"""

genomic = pd.read_csv(genomic_file, 
                      sep='\t', 
                      nrows=100)

genomic.columns.to_list()

['Virus name',
 'Type',
 'Accession ID',
 'Collection date',
 'Location',
 'Additional location information',
 'Sequence length',
 'Host',
 'Patient age',
 'Gender',
 'Clade',
 'Pango lineage',
 'Pangolin version',
 'Variant',
 'AA Substitutions',
 'Submission date',
 'Is reference?',
 'Is complete?',
 'Is high coverage?',
 'Is low coverage?',
 'N-Content',
 'GC-Content']

In [6]:
"""
Column Selection
----
Select the columns we want to keep, and rename them for convenience. 

columns_map: dict
    This dict maps original column names to desired column names.
    All other columns will not be imported. 
"""


columns_map = {
    'Virus name': 'virus_name',
    'Type': 'type',
    'Collection date': 'collection_date',
    'Location': 'location',
    'Sequence length': 'sequence_length',
    'Patient age': 'age',
    'Gender': 'sex',
    'Pango lineage': 'pango_lineage',
    #'Pangolin version': 'pangolin_version',
    'Variant': 'variant',
    'Submission date':'submitted_date', 
    'Is complete?': 'is_complete',
}


In [7]:
"""
Load genomic data from TSV file
----

GISAID provides TSV tab delimited format. 
We load the columns specified in the `columns_map` dict and rename accordingly. 
"""

# import genomic data into a dataframe
genomic = pd.read_csv(genomic_file, 
                      sep='\t', 
                      usecols = columns_map.keys(),
                      )


# rename the columns
genomic.rename(columns=columns_map, inplace=True)

/home/aman/anaconda3/envs/workspace/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
genomic.head(2).T

,0,1
virus_name,hCoV-19/Australia/NT12/2020,hCoV-19/Australia/NT13/2020
type,betacoronavirus,betacoronavirus
collection_date,2020-03-25,2020-03-25
location,Oceania / Australia / Northern Territory,Oceania / Australia / Northern Territory
sequence_length,29862,29865
age,unknown,unknown
sex,unknown,unknown
pango_lineage,B.1,B.1
variant,NaN,NaN
submitted_date,2020-04-17,2020-04-17


# Processing genomic data


Things we may want to do
* select region of interest
* select variant or lineage
* manage NaNs and dtypes


In [9]:
"""
select rows in our region of interest
---

Region needs to unambiguously find rows matching that location in GISAID

"""

# IMPORTANT:
# use all lower case when specifying the target region
# be as specific as possible 
# for example 'mexico' would catch 'new mexico'

target_region = 'north america / mexico'
# target_region = 'argentina'
# target_region = 'india'
# target_region = 'brazil'
# target_region = 'south africa'
# target_region = 'rio de janeiro'


# create boolean mask 
mask = genomic.location.str.lower().str.contains(target_region)

# notify
print (f"Original number of rows: {len(genomic)}; reduced to {mask.sum()} rows.")

# perform the horizontal slice
genomic = genomic[mask]

Original number of rows: 4439181; reduced to 32056 rows.


In [10]:
# sanity check
len(genomic)

32056

In [11]:
"""
Select rows based on lineage / variant. 
---

"""

pass

## Filter by pangolin_lineage (eg P.1) or variant name (eg gamma)

# target_lineage = 'B.1.1.7'
# target_variant = 'delta'


# other options

# target_variant = 'B.1.351'
# target_variant = 'P.1'
# target_variant = 'P'
# target_variant = 'B.1.617.2'

# target_variant = 'alpha'



In [12]:
"""Manage NaNs and dtypes
"""

# missing data is marked with "?". Convert to NaN
genomic.replace('?', np.nan, inplace=True)


## Convert types

# columns
columns = genomic.columns.tolist()

# datetime columns
datetimecols = [col for col in columns if 'date' in col]
print (f"Converting datetime columns: {datetimecols}")

for col in datetimecols: 
    genomic[col] = pd.to_datetime(genomic[col])


Converting datetime columns: ['collection_date', 'submitted_date']


In [13]:
genomic.dtypes

virus_name                 object
type                       object
collection_date    datetime64[ns]
location                   object
sequence_length             int64
age                        object
sex                        object
pango_lineage              object
variant                    object
submitted_date     datetime64[ns]
is_complete                object
dtype: object

## Inspect 

In [14]:
genomic.head(2).T

,414,421
virus_name,hCoV-19/Mexico/CMX-INER-02/2020,hCoV-19/Mexico/CMX-INER-01/2020
type,betacoronavirus,betacoronavirus
collection_date,2020-03-13 00:00:00,2020-03-12 00:00:00
location,North America / Mexico / Mexico City,North America / Mexico / Mexico City
sequence_length,29866,29864
age,55,42
sex,Female,Male
pango_lineage,B.1,B.1
variant,NaN,NaN
submitted_date,2020-04-13 00:00:00,2020-04-12 00:00:00
